In [252]:
using Random
using DataFrames
using AutoMLPipeline
using AutoMLPipeline.FeatureSelectors
using AutoMLPipeline.CrossValidators
using AutoMLPipeline.DecisionTreeLearners
using AutoMLPipeline.Pipelines
using AutoMLPipeline.BaseFilters
using AutoMLPipeline.SKPreprocessors
using AutoMLPipeline.EnsembleMethods
using AutoMLPipeline.Utils
ENV["COLUMNS"]=1000; # for dataframe column size

In [2]:
using CSV

In [96]:
data=CSV.read("profb.csv")
first(data,10)

,Home.Away,Favorite_Points,Underdog_Points,Pointspread,Favorite_Name,Underdog_name,Year
,String,Int64,Int64,Float64,String,String,Int64
1,away,27,24,4.0,BUF,MIA,89
2,at_home,17,14,3.0,CHI,CIN,89
3,away,51,0,2.5,CLE,PIT,89
4,at_home,28,0,5.5,NO,DAL,89
5,at_home,38,7,5.5,MIN,HOU,89
6,at_home,34,20,6.0,DEN,KC,89
7,away,31,21,6.0,LAN,ATL,89
8,at_home,24,27,2.5,NYJ,NE,89
9,away,16,13,1.5,PHX,DET,89


In [98]:
Y = data[:,1] |> collect
X = data[:,2:end];

In [274]:
# decomposition
pca = SKPreprocessor("PCA")
fa = SKPreprocessor("FactorAnalysis")
ica = SKPreprocessor("FastICA")
# Scaler 
rb = SKPreprocessor("RobustScaler")
pt = SKPreprocessor("PowerTransformer")
norm = SKPreprocessor("Normalizer")
mx = SKPreprocessor("MinMaxScaler")
# categorical preprocessing
ohe = OneHotEncoder()
# Column selector
disc = CatNumDiscriminator()
catf = CatFeatureSelector()
numf = NumFeatureSelector()
# Learners
rf = SKLearner("RandomForestClassifier")
gb = SKLearner("GradientBoostingClassifier")
lsvc = SKLearner("LinearSVC")
svc = SKLearner("SVC")
mlp = SKLearner("MLPClassifier")
ada = SKLearner("AdaBoostClassifier");
jrf = RandomForest();
vote = VoteEnsemble();
stack = StackEnsemble();
best = BestLearner();

In [281]:
p1 = @pipeline  (catf+ohe)*(numf) + vote
pred = fit_transform!(p1,X,Y)
score(:accuracy,pred,Y)

100.0

In [283]:
p1 = @pipeline  (numf+rb+pca) * (numf+rb+ica) * (catf+ohe) * (numf+rb+fa) + rf
pred = fit_transform!(p1,X,Y)
score(:accuracy,pred,Y)

97.47023809523809

In [284]:
crossvalidate(p1,X,Y,"accuracy_score",5)

fold: 1, 0.6716417910447762
fold: 2, 0.6296296296296297
fold: 3, 0.6194029850746269
fold: 4, 0.7481481481481481
fold: 5, 0.6343283582089553


(mean = 0.6606301824212272, std = 0.05275687604661152, folds = 5)

In [285]:
p2 = @pipeline (numf+rb+pca) * (numf+rb+ica)  * (numf+rb+fa) + gb
pred = fit_transform!(p2,X,Y)
score(:accuracy,pred,Y)

87.94642857142857

In [190]:
crossvalidate(p2,X,Y,"accuracy_score",5)

fold: 1, 0.5895522388059702
fold: 2, 0.6
fold: 3, 0.6268656716417911
fold: 4, 0.6296296296296297
fold: 5, 0.582089552238806


(mean = 0.6056274184632394, std = 0.021629146567337453, folds = 5)